# The Battle of Neighborhoods (Week 1)

## Introduction

This project helps to recommend a suitable place for opening an Italian restaurant. We need to find places that will be best suited for opening an Italian restaurant in New Delhi. Since there are lots of restaurants in Delhi, we will try to detect locations that are not already crowded with restaurants. We are also particularly interested in areas with no Italian restaurants in vicinity.

We will explore the neighborhoods and find the most suitable area suited for our restaurant based on the above criteria. Advantages of each area will be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data Description

The data required for finding a suitable location for our restaurant will be data of places that are less crowded by Indian Restaurants. For this purpose, we need to collect this data on all the neighborhood of Delhi. Dataset used in this project is taken from Kaggle. (https://www.kaggle.com/shaswatd673/delhi-neighborhood-data)

Once we have collected all the data the places will be ranked on the basis of nearby venues that contribute to our target audience. Based on definition of our problem, factors that will influence our decision are:
1. Number of restaurants in a neighborhood
2. Number of Italian restaurants in a neighborhood

Coordinates of the centre of New Delhi, and of all the neighborhoods are obtained using Here API.

Number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API.